# Load AutoLaparo

In [ ]:
import os
import sys
sys.path.append("../..")

import pandas as pd


from utils.io import load_yaml

seq_len = 15
increment = 5

server = "local"
server = load_yaml("../../config/servers.yml")[server]
database = "autolaparo_single_frames/AutoLaparo_Task2"
prefix = os.path.join(server["database"]["location"], database)
pkl = f"23_03_03_motion_label_window_1_frame_increment_{increment}_frames_between_clips_1_log_test_train.pkl"
# pkl = f"23_03_03_motion_label_window_1_frame_increment_{increment}_frames_between_clips_1_log.pkl"
# pkl = f"23_03_03_pre_processed_frame_increment_{increment}_frames_between_clips_1_log.pkl"


vid_df = pd.read_pickle(os.path.join(prefix, pkl))

# sort!
vid_df[["vid", "frame"]] = vid_df[["vid", "frame"]].astype(float)
vid_df = vid_df.sort_values(by=["vid", "frame"]).reset_index(drop=True)
vid_df[["vid", "frame"]] = vid_df[["vid", "frame"]].astype(int)

length = len(vid_df[vid_df["vid"] == 0])

# get last half plus sequence length
# print(len(vid_df)) 250 frames, of which we take half. Increment is missing due to pre-processing
vid_df = vid_df.groupby("vid").tail(125 -increment + seq_len ).reset_index(drop=True)
# print(len(vid_df))

label_df = pd.read_csv(os.path.join(prefix, "laparoscope_motion_label.csv"))

print(vid_df)
print(label_df)

# from readme
motion_dict = {
    0: "Static",
    1: "Up",
    2: "Down",
    3: "Left",
    4: "Right",
    5: "Zoom-in",
    6: "Zoom-out",
}

# Load Homography Predictor

In [ ]:
import os
import sys
sys.path.append("../..")
import pandas as pd
import numpy as np
from kornia import tensor_to_image
from torch.utils.data import DataLoader
import PIL
from IPython.display import display, clear_output

from utils.io import load_yaml
from datasets import ImageSequenceDataset


server = "local"
server = load_yaml("../../config/servers.yml")[server]
database = "autolaparo_single_frames/AutoLaparo_Task2"
prefix = os.path.join(server["database"]["location"], database)
pkl = "log.pkl"

df = pd.read_pickle(os.path.join(prefix, pkl))
df_a = df[df["vid"] == 298]
df_b = df[df["vid"] == 299]

df = pd.concat([df_a, df_b]).reset_index(drop=True)

ds = ImageSequenceDataset(
    df,
    prefix,
    seq_len=10,
    frame_increment=5,
    frames_between_clips=50,
)

ds._df = ds._df.astype(object)
dl = DataLoader(
    ds,
    num_workers=0,
    batch_size=1,
    drop_last=False,
    shuffle=False,
)

from kornia.geometry import resize

for batch in dl:
    imgs, tf_imgs, frame_idcs, vid_idcs = batch
    imgs = imgs.float()/255.
    imgs = resize(imgs, (240, 320))
    print(imgs.shape)
    # print(frame_idcs)

    # for img in imgs[0]: # batch 0
    #     img = tensor_to_image(img, keepdim=False).astype(np.uint8)
    #     display(PIL.Image.fromarray(img))
    #     clear_output(wait=True)




In [ ]:
from lightning_modules.homography_imitation import ConvHomographyPredictorModule


checkpoint_prefix = "/tmp/miccai/cholec80/resnet18/version_0"
checkpoint = "checkpoints/epoch=65-step=129360.ckpt"

config = load_yaml(os.path.join(checkpoint_prefix, "config.yml"))

module = ConvHomographyPredictorModule.load_from_checkpoint(
    os.path.join(checkpoint_prefix, checkpoint), **config["model"]
)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from kornia import tensor_to_image
from torch.utils.data import DataLoader
from datasets import ImageSequenceDataset

from utils.viz import create_blend_from_four_point_homography
from utils.transforms import dict_list_to_augment

import PIL
from IPython.display import clear_output, display

transforms = dict_list_to_augment([
    {"chance": 1.0, "module": "imgaug.augmenters", "type": "Resize", "kwargs": {"size": {"height": 240, "width": 320}}}
])

ds = ImageSequenceDataset(
    df=vid_df,
    prefix=prefix,
    seq_len=15,
    frame_increment=10,
    frames_between_clips=100,
    geometric_transforms=transforms,
)

dl = DataLoader(ds, batch_size=1, shuffle=False, num_workers=0)

for batch in dl:
    imgs, imgs_tf, frame_idcs, vid_idx = batch
    B, T, C, H, W = imgs.shape
    preview_horizon = imgs[:, :-1]
    preview_horizon = preview_horizon.float() / 255.
    preview_horizon = preview_horizon.reshape(B, -1, H, W)

    duvs = module(preview_horizon)
    print(duvs)
    
    # for img in imgs[0]: # batch 0
    #     img = (tensor_to_image(img, keepdim=False)*255.).astype(np.uint8)
    #     display(PIL.Image.fromarray(img))
    #     clear_output(wait=True)

    break

    # input = imgs[:, :-1]
    # input = input.reshape(B, -1, H, W)
    # duv = module(input)*100

    # # direction lsos
    # blend = create_blend_from_four_point_homography(imgs[:, -2], imgs[:, -1], duv)
    # blend = (tensor_to_image(blend, keepdim=False) * 255.).astype(np.uint8)
    
    # blend = cv2.resize(blend, (640, 480))
    # cv2.imwrite(f"/tmp/chin/dara/{idx}.png", blend)
    # idx += 1
    # if idx >= 100:
    #     break

